#### BIBLIOTECAS

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime

#### LENDO OS DADOS

In [5]:
df = pd.read_json(
    r'C:\Users\marco\OneDrive\Documentos\Projetos\Python\Web Scraping - Mercado Livre\coleta\..dados\bd.jsonl', lines=True)

In [10]:
df.head()

,marca,nome,preco_antigo_reais,preco_antigo_centavos,preco_novo_reais,preco_novo_centavos,reviews_rating_number,reviews_amount
0,OLYMPIKUS,Olympikus Express Masculino Adultos,274.000,99,217,79.0,4.7,(241)
1,OLYMPIKUS,Olympikus 24/7 Lap Masculino Adultos,218.000,74,139,99.0,4.8,(7829)
2,REEBOK,Reebok Zig Dynamica 4 Masculino Adultos,1.099,64,106,NaN,NaN,None
3,OLYMPIKUS,Olympikus 24/7 Lap Masculino Adultos,218.000,74,139,99.0,4.8,(7829)
4,REEBOK,Reebok Zig Dynamica 4 Masculino Adultos,1.099,64,106,NaN,NaN,None


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   marca                  50 non-null     object 
 1   nome                   54 non-null     object 
 2   preco_antigo_reais     54 non-null     float64
 3   preco_antigo_centavos  54 non-null     int64  
 4   preco_novo_reais       54 non-null     int64  
 5   preco_novo_centavos    49 non-null     float64
 6   reviews_rating_number  27 non-null     float64
 7   reviews_amount         27 non-null     object 
dtypes: float64(3), int64(2), object(3)
memory usage: 3.5+ KB


#### TRATANDO OS DADOS

In [12]:
# Tratar os valores nulos para colunas numéricas e de texto
df['preco_antigo_reais'] = df['preco_antigo_reais'].fillna(0).astype(float)
df['preco_antigo_centavos'] = df['preco_antigo_centavos'].fillna(
    0).astype(float)
df['preco_novo_reais'] = df['preco_novo_reais'].fillna(0).astype(float)
df['preco_novo_centavos'] = df['preco_novo_centavos'].fillna(0).astype(float)
df['reviews_rating_number'] = df['reviews_rating_number'].fillna(
    0).astype(float)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   marca                  50 non-null     object 
 1   nome                   54 non-null     object 
 2   preco_antigo_reais     54 non-null     float64
 3   preco_antigo_centavos  54 non-null     float64
 4   preco_novo_reais       54 non-null     float64
 5   preco_novo_centavos    54 non-null     float64
 6   reviews_rating_number  54 non-null     float64
 7   reviews_amount         27 non-null     object 
dtypes: float64(5), object(3)
memory usage: 3.5+ KB


In [14]:
# Tratar os preços como floats e calcular os valores totais
df['preco_antigo'] = df['preco_antigo_reais'] + \
    df['preco_antigo_centavos'] / 100
df['preco_novo'] = df['preco_novo_reais'] + df['preco_novo_centavos'] / 100

In [15]:
# Remover os parênteses das colunas `reviews_amount`
df['reviews_amount'] = df['reviews_amount'].str.replace(
    '[\(\)]', '', regex=True)
df['reviews_amount'] = df['reviews_amount'].fillna(0).astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\marco\AppData\Local\Temp\ipykernel_8452\308019279.py:2: SyntaxWarning: invalid escape sequence '\('
  df['reviews_amount'] = df['reviews_amount'].str.replace('[\(\)]', '', regex=True)


In [16]:
# Adicionar a coluna de origem dos dados
df['fonte'] = "https://lista.mercadolivre.com.br/tenis-corrida-masculino"

In [17]:
# Adicionar a coluna _data_coleta com a data e hora atuais
df['data_coleta'] = datetime.now()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   marca                  50 non-null     object        
 1   nome                   54 non-null     object        
 2   preco_antigo_reais     54 non-null     float64       
 3   preco_antigo_centavos  54 non-null     float64       
 4   preco_novo_reais       54 non-null     float64       
 5   preco_novo_centavos    54 non-null     float64       
 6   reviews_rating_number  54 non-null     float64       
 7   reviews_amount         54 non-null     int32         
 8   preco_antigo           54 non-null     float64       
 9   preco_novo             54 non-null     float64       
 10  fonte                  54 non-null     object        
 11  data_coleta            54 non-null     datetime64[us]
dtypes: datetime64[us](1), float64(7), int32(1), object(3)
memory usage

In [22]:
# Remover as colunas antigas de preços
df.drop(columns=['preco_antigo_reais', 'preco_antigo_centavos',
        'preco_novo_reais', 'preco_novo_centavos'], inplace=True)

#### SALVANDO NO BANCO DE DADOS

In [23]:
# Conectar ao banco de dados SQLite (ou criar um novo)
conn = sqlite3.connect(
    r'C:\Users\marco\OneDrive\Documentos\Projetos\Python\Web Scraping - Mercado Livre\coleta\..dados\bd.db')

In [24]:
# Salvar o DataFrame no banco de dados SQLite
df.to_sql('mercadolivre_items', conn, if_exists='replace', index=False)

54

In [25]:
# Fechar a conexão com o banco de dados
conn.close()